In [1]:
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

c:\Users\nishi\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_excel("./Dataset.xlsx")
data = data.rename(columns={'Sentence': 'text', 'Label': 'label'})
data

,text,label
0,The development of reinforcement techniques in...,a1
1,SERMO asks the user on a daily basis on events...,a2
2,This finding renders the IL-Cl passivated pero...,a5
3,"In this paper, an investigation has been carri...",a3
4,"In this survey, we perform a broad and thoroug...",a3
...,...,...
246,This paper describes methods for explanatory a...,a2
247,Using annual time series data from 1971 to 201...,a3
248,Findings in this work offer strong support to ...,a5
249,The short carbon‐fibre‐reinforced PolyEtherKet...,a3


In [4]:
print([data[data['label'] == label].shape for label in data['label'].unique()]) # this is to check if the data is balanced or not, as in if the number of samples for each class is the same or not

[(44, 2), (42, 2), (44, 2), (76, 2), (45, 2)]


In [5]:
from sentence_transformers.losses import CosineSimilarityLoss
from datasets import Dataset, Features
from setfit import SetFitModel, SetFitTrainer

In [6]:
features = Features.from_dict({'text': {'dtype': 'string', 'id': None, '_type': 'Value'}, 'label': {'names': ['a1', 'a2', 'a3', 'a4', 'a5'], 'id': None, '_type': 'ClassLabel'}})
train = data.groupby('label').head(20).reset_index(drop=True)
test = data.groupby('label').head(-20).reset_index(drop=True)
train = Dataset.from_pandas(train, features=features)
test = Dataset.from_pandas(test, features=features)
print(train, test)

Dataset({
    features: ['text', 'label'],
    num_rows: 100
}) Dataset({
    features: ['text', 'label'],
    num_rows: 151
})


In [ ]:
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=test,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

trainer.train()

In [93]:
trainer.evaluate()

***** Running evaluation *****


{'accuracy': 0.7615894039735099}

to do: shuffle and take accuracy score on avg